# Lab 25: DFIR Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab25_dfir_fundamentals.ipynb)

Essential incident response concepts before diving into advanced DFIR labs.

## Learning Objectives
- Understand the incident response lifecycle
- Identify common attack artifacts (processes, files, network, registry)
- Map findings to MITRE ATT&CK techniques
- Build an artifact analysis toolkit

**Next:** Lab 29 (IR Copilot)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q pandas

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass
from datetime import datetime

print("✅ Libraries loaded!")

## Attack Artifacts: What to Look For

### Process Artifacts

| Indicator | Why It's Suspicious | Example |
|-----------|---------------------|---------|
| **Unusual parent** | Office spawning cmd/powershell | WINWORD.EXE → powershell.exe |
| **Wrong path** | System binary from user folder | C:\Users\Public\svchost.exe |
| **Encoded commands** | Obfuscation attempt | powershell -enc JABjAG... |
| **No arguments** | Possible injected process | rundll32.exe (no DLL) |

### Network Artifacts

| Indicator | Pattern | Technique |
|-----------|---------|-----------|
| **Beaconing** | Regular interval connections | C2 communication |
| **DNS tunneling** | Long subdomains, high volume | Data exfiltration |
| **Unusual ports** | 443 to non-HTTPS server | Encrypted C2 |

In [ ]:
# Process Anomaly Detection

@dataclass
class ProcessInfo:
    """Represents a running process."""
    pid: int
    name: str
    path: str
    parent_name: str
    command_line: str

# Suspicious parent-child relationships
SUSPICIOUS_PARENTS = {
    "powershell.exe": ["winword.exe", "excel.exe", "outlook.exe", "mshta.exe"],
    "cmd.exe": ["winword.exe", "excel.exe", "outlook.exe", "mshta.exe"],
    "rundll32.exe": ["powershell.exe", "cmd.exe", "wscript.exe"],
    "regsvr32.exe": ["powershell.exe", "cmd.exe"],
}

# System binaries that should be in system folders
SYSTEM_BINARIES = ["svchost.exe", "csrss.exe", "lsass.exe", "services.exe", "smss.exe"]
SYSTEM_PATHS = ["c:\\windows\\system32", "c:\\windows\\syswow64"]

def analyze_process(proc: ProcessInfo) -> List[str]:
    """
    Analyze a process for suspicious indicators.

    Args:
        proc: Process information

    Returns:
        List of suspicious findings
    """
    findings = []
    proc_name = proc.name.lower()
    parent_name = proc.parent_name.lower()
    path = proc.path.lower()
    cmd = proc.command_line.lower()

    # Check 1: Suspicious parent-child
    if proc_name in SUSPICIOUS_PARENTS:
        if parent_name in SUSPICIOUS_PARENTS[proc_name]:
            findings.append(f"🔴 Suspicious parent: {parent_name} → {proc_name}")

    # Check 2: System binary masquerading
    if proc_name in SYSTEM_BINARIES:
        if not any(sp in path for sp in SYSTEM_PATHS):
            findings.append(f"🔴 Masquerading: {proc_name} running from {path}")

    # Check 3: Encoded PowerShell
    if "powershell" in proc_name and ("-enc" in cmd or "-encoded" in cmd):
        findings.append("🟡 Encoded PowerShell command detected")

    # Check 4: Living off the land binaries (LOLBins)
    lolbins = ["certutil", "bitsadmin", "mshta", "regsvr32", "rundll32"]
    if any(lol in proc_name for lol in lolbins) and ("http" in cmd or "ftp" in cmd):
        findings.append(f"🔴 LOLBin with network activity: {proc_name}")

    return findings

# Test with sample processes
test_processes = [
    ProcessInfo(1234, "powershell.exe", "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
                "winword.exe", "powershell.exe -enc JABjAGwAaQBlAG4AdA..."),
    ProcessInfo(5678, "svchost.exe", "C:\\Users\\Public\\svchost.exe",
                "cmd.exe", "svchost.exe"),
    ProcessInfo(9012, "chrome.exe", "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe",
                "explorer.exe", "chrome.exe --profile-directory=Default"),
]

print("🔍 PROCESS ANALYSIS")
print("=" * 60)

for proc in test_processes:
    findings = analyze_process(proc)
    if findings:
        print(f"\n⚠️ {proc.name} (PID {proc.pid}):")
        for f in findings:
            print(f"   {f}")
    else:
        print(f"\n✅ {proc.name} (PID {proc.pid}): No anomalies")

## MITRE ATT&CK Mapping

Every finding should map to ATT&CK techniques!

In [ ]:
# MITRE ATT&CK Mapping

ATTACK_TECHNIQUES = {
    "T1059.001": {
        "name": "PowerShell",
        "tactic": "Execution",
        "patterns": ["powershell", "-enc", "-encoded", "invoke-expression"],
    },
    "T1055": {
        "name": "Process Injection",
        "tactic": "Defense Evasion",
        "patterns": ["virtualalloc", "writeprocessmemory", "createremotethread"],
    },
    "T1003.001": {
        "name": "LSASS Memory",
        "tactic": "Credential Access",
        "patterns": ["mimikatz", "sekurlsa", "lsass"],
    },
    "T1547.001": {
        "name": "Registry Run Keys",
        "tactic": "Persistence",
        "patterns": ["currentversion\\run", "runonce", "hklm\\software\\microsoft\\windows"],
    },
    "T1071.001": {
        "name": "Web Protocols",
        "tactic": "Command and Control",
        "patterns": ["beacon", "c2", "http://", "https://"],
    },
    "T1486": {
        "name": "Data Encrypted for Impact",
        "tactic": "Impact",
        "patterns": ["ransomware", "encrypt", ".locked", "ransom"],
    },
    "T1036.005": {
        "name": "Masquerading",
        "tactic": "Defense Evasion",
        "patterns": ["svchost", "lsass", "csrss"],  # When in wrong path
    },
}

def map_to_attack(finding: str) -> List[Dict]:
    """
    Map a finding to MITRE ATT&CK techniques.

    Args:
        finding: Description of the finding

    Returns:
        List of matching techniques
    """
    matches = []
    finding_lower = finding.lower()

    for tech_id, tech_info in ATTACK_TECHNIQUES.items():
        if any(pattern in finding_lower for pattern in tech_info["patterns"]):
            matches.append({
                "id": tech_id,
                "name": tech_info["name"],
                "tactic": tech_info["tactic"],
            })

    return matches

# Test ATT&CK mapping
test_findings = [
    "PowerShell execution with -enc parameter detected",
    "Suspicious process svchost.exe running from C:\\Users\\Public",
    "Registry key modified: HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run",
    "Outbound connection to http://evil-c2.com/beacon",
]

print("🎯 MITRE ATT&CK MAPPING")
print("=" * 60)

for finding in test_findings:
    print(f"\nFinding: {finding[:50]}...")
    techniques = map_to_attack(finding)
    if techniques:
        for tech in techniques:
            print(f"   → {tech['id']}: {tech['name']} ({tech['tactic']})")
    else:
        print("   → No technique mapping found")

## 🎉 Key Takeaways

1. **Follow the lifecycle** - Preparation → Identification → Containment → Eradication → Recovery → Lessons Learned
2. **Know your artifacts** - Processes, files, network, registry all tell a story
3. **Map to ATT&CK** - Gives common language and helps identify gaps
4. **Preserve evidence** - Order of volatility matters (memory first!)
5. **Document everything** - Chain of custody is critical

## Severity Levels

| Level | Meaning | Response Time |
|-------|---------|---------------|
| 🔴 CRITICAL | Active threat | < 15 minutes |
| 🟠 HIGH | Likely malicious | < 1 hour |
| 🟡 MEDIUM | Suspicious | < 4 hours |
| 🟢 LOW | Informational | < 24 hours |

## Next Steps

- **Lab 31**: Ransomware Detection
- **Lab 13**: Memory Forensics with AI
- **Lab 14**: C2 Traffic Analysis